In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sn

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, accuracy_score
from sklearn.pipeline import Pipeline
import joblib

%matplotlib inline

In [19]:
 class CustomerSupportAutomation:
    def __init__(self):
        # Initialize the machine learning model pipeline and predefined responses
        self.model_pipeline = None
        self.responses = {
            "cancel_order": "Sure, I can assist with canceling your order. Please provide your order number.",
            "change_order": "Absolutely, let me help you modify your order. What changes would you like to make?",
            "change_shipping_address": "I can help with updating your shipping address. Please share your order number and the new address.",
            "check_cancellation_fee": "I can check the cancellation fee for you. Which service or product is this regarding?",
            "check_invoice": "Let me assist you with your invoice. Could you provide your account or order number?",
            "check_payment_methods": "We accept credit cards, PayPal, and bank transfers. Which payment method do you need more information on?",
            "check_refund_policy": "Our refund policy generally offers full refunds within 30 days of purchase. Would you like more specifics?",
            "complaint": "I'm sorry you're experiencing issues. Please provide more details about your complaint so we can address it.",
            "contact_customer_service": "You are connected to customer service. How can I assist you?",
            "contact_human_agent": "I understand you wish to speak with a human agent. Transferring you now, please hold.",
            "create_account": "I can help you create an account. Please provide your email address to get started.",
            "delete_account": "I can assist with deleting your account. May I know the reason for deletion? We might be able to resolve your concerns.",
            "delivery_options": "We offer standard, express, and next-day delivery options. Which one would you like to know more about?",
            "delivery_period": "Delivery times vary by location and shipping method. Please provide your zip code and the shipping method.",
            "edit_account": "I can help you edit your account details. What would you like to update?",
            "get_invoice": "I can retrieve your invoice. Please provide your order number or purchase date.",
            "get_refund": "You can request a refund. Please provide your order number and the reason for the refund.",
            "newsletter_subscription": "I can assist with your newsletter subscription. Would you like to subscribe or unsubscribe?",
            "payment_issue": "I'm sorry you're facing a payment issue. Please describe the problem in more detail.",
            "place_order": "I can help you place an order. Which product or service are you interested in?",
            "recover_password": "I can assist with password recovery. Please provide the email address associated with your account.",
            "registration_problems": "I'm sorry you're having registration issues. Please describe the problem you're encountering.",
            "review": "We value your feedback. Would you like to leave a review for a specific product or your overall experience?",
            "set_up_shipping_address": "I can help you set up your shipping address. Please provide your full address including street, city, state, and zip code.",
            "switch_account": "I can assist with switching accounts. Please verify your current account information and the details of the new account.",
            "track_order": "I can help you track your order. Please provide your order number.",
            "track_refund": "I can help you track your refund. Please provide the order number for your refund request."
        }

    def load_and_combine_datasets(self, file_paths):
        # Initialize an empty list to store the dataframes
        dataframes = []
    
        # Iterate through the file paths and read each CSV file into a dataframe
        for file_path in file_paths:
            df = pd.read_csv(file_path)
            dataframes.append(df)
       
        # Concatenate all the dataframes into a single dataframe
        combined_data = pd.concat(dataframes, axis=0, ignore_index=True)
        return combined_data

    def preprocess_text(self, df):
        # Apply a lambda function to convert text to lowercase
        df['utterance'] = df['utterance'].apply(lambda x: x.lower())
        return df

    def train_model(self, df):
        # Define features and labels
        features = df['utterance']
        labels = df['intent']
    
        # Split the data into training and testing sets
        X_train, X_test, y_train, y_test = train_test_split(
            features, labels, test_size=0.2, random_state=42)

        # Create the TF-IDF vectorizer
        tfidf_vectorizer = TfidfVectorizer(max_features=5000)

        # Create the Naive Bayes classifier
        nb_classifier = MultinomialNB()

        # Build the model pipeline
        self.model_pipeline = Pipeline([
            ('vectorizer', tfidf_vectorizer),
            ('classifier', nb_classifier)
        ])

        # Fit the model to the training data
        self.model_pipeline.fit(X_train, y_train)

        # Generate predictions for the test dataset
        predictions = self.model_pipeline.predict(X_test)

        # Calculate the accuracy and display evaluation metrics
        accuracy = accuracy_score(y_test, predictions)
        print(f"Model Accuracy: {accuracy:.2f}")
        print(classification_report(y_test, predictions))
    
    def save_model(self, filename):
        #Serialize the trained model to the specified file.
        # Use joblib to save the model pipeline
        joblib.dump(self.model_pipeline, filename)

    def load_model(self, filename):
        #Load a pre-trained model from the specified file.
        # Deserialize the model pipeline using joblib
        self.model_pipeline = joblib.load(filename)

    def predict_user_intent(self, utterance):
        #Determine the user's intent based on the provided utterance.
        # Predict the intent using the model pipeline
        predicted_intent = self.model_pipeline.predict([utterance])
        return predicted_intent[0]

    def generate_response(self, utterance):
        #Create a response based on the identified user intent.
        # Determine the user's intent
        intent = self.predict_user_intent(utterance)
        # Fetch the corresponding response or a fallback message
        return self.responses.get(intent, "I apologize, I didn't catch that. Could you please clarify?")
 


In [20]:
def main():
    # Initialize the customer support bot
    bot = CustomerSupportAutomation()

    # List of dataset file paths
    dataset_paths = [
        '21534-Bitext-sample-Training-dataset-for-chatbots-virtual-assistant.csv',
        'Bitext_Sam_Cus_Ser_Training_Dataset.csv'
    ]

    # Load and preprocess the datasets
    combined_data = bot.load_and_combine_datasets(dataset_paths)
    cleaned_data = bot.preprocess_text(combined_data)

    # Train the model with the preprocessed data
    bot.train_model(cleaned_data)

    # Save the trained model to a file
    bot.save_model('customer_support_model.joblib')

    # Define sample queries for testing the bot
    sample_queries = [
        "How can I track my package?",
        "I would like to change my address",
        "What payment options do you offer?",
        "Help me with my password, please.",
        "May I speak to a human agent?",
        "How can I switch accounts?"
    ]

    # Iterate over the sample queries and print responses
    for query in sample_queries:
        response = bot.generate_response(query)
        print(f"Query: {query}\nResponse: {response}\n")

if __name__ == "__main__":
    main()


Model Accuracy: 0.97
                          precision    recall  f1-score   support

            cancel_order       1.00      0.72      0.84        80
            change_order       0.94      0.99      0.97       258
 change_shipping_address       1.00      0.93      0.97        76
  check_cancellation_fee       1.00      1.00      1.00       126
           check_invoice       1.00      0.62      0.77        48
          check_invoices       0.97      0.81      0.88       198
   check_payment_methods       1.00      0.88      0.94       124
     check_refund_policy       1.00      0.99      1.00       157
               complaint       0.99      0.99      0.99       192
contact_customer_service       0.98      1.00      0.99       473
     contact_human_agent       1.00      0.97      0.99       273
          create_account       0.97      1.00      0.98       436
          delete_account       0.98      1.00      0.99       260
        delivery_options       0.98      0.99      0.9